# Import Libraries

In [54]:
import os

# Path where you want to create the folder
folder_path = '/content/drive/My Drive/Colab Notebooks/Images_detection'

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created:", folder_path)
else:
    print("Folder already exists.")


Folder already exists.


In [ ]:
from google.colab import files

uploaded = files.upload()

for file_name in uploaded.keys():
    print('Uploaded file:', file_name)


In [105]:
import shutil
import os

# Get the name of the uploaded video file
video_name = list(uploaded.keys())[0]

# Path to the folder in your Google Drive
folder_path = '/content/drive/My Drive/Colab Notebooks/Images_detection/'

# Move the uploaded video to the specified folder
shutil.move(video_name, os.path.join(folder_path, video_name))

print("Video moved to Google Drive folder.")


Video moved to Google Drive folder.


In [ ]:
!pip install mtcnn

In [ ]:
# import cv2
# from mtcnn import MTCNN
# import os
# from google.colab.patches import cv2_imshow

# # Load the video file
# video_path = "/content/drive/MyDrive/Colab Notebooks/Images_detection/sa1-video-felc0.avi"
# cap = cv2.VideoCapture(video_path)

# # Create MTCNN detector
# detector = MTCNN()

# # Set the time limit
# end_time = cv2.getTickCount() + 30 * cv2.getTickFrequency()  # 30 seconds

# # Create a folder to save images
# output_folder = "/content/drive/MyDrive/Colab Notebooks/Images_detection"
# os.makedirs(output_folder, exist_ok=True)

# while cv2.waitKey(1) != ord('x'):
#     if cv2.getTickCount() > end_time:
#         break

#     ret, frame = cap.read()
#     if not ret:
#         print("Error reading frame.")
#         break

#     # Detect faces
#     output = detector.detect_faces(frame)
#     print("Number of faces detected:", len(output))

#     for i, single_output in enumerate(output):
#         x, y, width, height = single_output['box']
#         face = frame[y:y + height, x:x + width]

#         # Save the face
#         cv2.imwrite(os.path.join(output_folder, f"face_{i}.jpg"), face)

#         # Draw rectangle around the face
#         cv2.rectangle(frame, pt1=(x, y), pt2=(x + width, y + height), color=(255, 0, 0), thickness=3)


#     # Show the frame
#     cv2_imshow(frame)

# # Release video capture and close windows
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
import cv2
from mtcnn import MTCNN
import os

# Load the video file
video_path = "/content/drive/MyDrive/Colab Notebooks/Images_detection/aawvkuxypy.mp4"
cap = cv2.VideoCapture(video_path)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error opening video file.")
    exit()

# Create MTCNN detector
detector = MTCNN()

# Set the time limit
end_time = cv2.getTickCount() + 30 * cv2.getTickFrequency()  # 30 seconds

# Create a folder to save images
output_folder = "/content/drive/MyDrive/Colab Notebooks/Images_detection"
os.makedirs(output_folder, exist_ok=True)

frame_count = 0
while True:
    if cv2.waitKey(1) == ord('x') or cv2.getTickCount() > end_time:
        break

    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Detect faces
    output = detector.detect_faces(frame)
    print("Number of faces detected:", len(output))

    for i, single_output in enumerate(output):
        x, y, width, height = single_output['box']
        face = frame[y:y + height, x:x + width]

        # Save the face
        cv2.imwrite(os.path.join(output_folder, f"face_{frame_count}_{i}.jpg"), face)

        # Draw rectangle around the face
        cv2.rectangle(frame, pt1=(x, y), pt2=(x + width, y + height), color=(255, 0, 0), thickness=3)

    # Save the frame
    # cv2.imwrite(os.path.join(output_folder, f"frame_{frame_count}.jpg"), frame)
    frame_count += 1
    cv2_imshow(frame)

# Release video capture
cap.release()

print("Faces saved in:", output_folder)


In [107]:
import os

def delete_extracted_images(image_output_folder):
    """Delete only image files in the image output folder."""
    for filename in os.listdir(image_output_folder):
        file_path = os.path.join(image_output_folder, filename)
        if os.path.isfile(file_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                try:
                    os.remove(file_path)
                    print(f"Deleted {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")





In [108]:
image_output_folder = "/content/drive/MyDrive/Colab Notebooks/Images_detection"


In [ ]:
pip install gradio

In [ ]:
pip install facenet-pytorch

In [ ]:
pip install grad-cam

In [11]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import warnings
warnings.filterwarnings("ignore")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download and Load Model

In [12]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()

In [13]:
file_path = "/content/drive/MyDrive/Colab Notebooks/resnetinceptionv1_epoch_32.pth"



In [ ]:
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

file_path = "/content/drive/MyDrive/Colab Notebooks/resnetinceptionv1_epoch_32.pth"

checkpoint = torch.load(file_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

# Model Inference

In [15]:
import matplotlib.pyplot as plt

In [109]:
def predict(input_image:Image.Image):
    """Predict the label of the input_image"""
    # print("Starting prediction...")

    # Face detection
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')
    # print("Face detected.")

    # Preprocessing
    face = face.unsqueeze(0)  # Add the batch dimension
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)
    # print("Image preprocessed.")

    # Visualization
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    prev_face = prev_face.astype('uint8')

    face = face.to(DEVICE)
    face = face.to(torch.float32)
    face = face / 255.0
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    # print("Visualization prepared.")

    # Grad-CAM
    target_layers = [model.block8.branch1[-1]]
    use_cuda = True if torch.cuda.is_available() else False
    cam = GradCAM(model=model, target_layers=target_layers)
    targets = [ClassifierOutputTarget(0)]

    grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
    face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)
    # print("Grad-CAM visualization created.")

    # Model prediction
    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        prediction = "real" if output.item() < 0.5 else "fake"

        real_prediction = 1 - output.item()
        fake_prediction = output.item()

        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }
        # print("Prediction made.")

    print("Prediction complete.")

    return confidences, face_with_mask



In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Path to the output folder
output_folder = "/content/drive/MyDrive/Colab Notebooks/Images_detection"

# Get a list of all image files in the output folder
image_files = [f for f in os.listdir(output_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

# List to store all predictions
predictions = []

for image_file in image_files:
    # Open the image
    image_path = os.path.join(output_folder, image_file)
    image = Image.open(image_path)

    # Call the predict function
    # confidences, _ = predict(image)
    confidences, image_with_mask = predict(image)
    plt.imshow(image_with_mask)
    plt.title(f"Image: {image_file} with Mask Overlay")
    plt.axis('off')
    plt.show()

    # Store the prediction
    if confidences['real'] > confidences['fake']:
        predictions.append("real")
    else:
        predictions.append("fake")

# Count the votes
vote_count = {
    "real": predictions.count("real"),
    "fake": predictions.count("fake")
}

# Determine the final result
if vote_count["real"] > vote_count["fake"]:
    final_result = "Real"
else:
    final_result = "Fake"

# Print vote count and final result
    # Display the image with the mask overlay

print("Vote Count:")
print("Real:", vote_count["real"])
print("Fake:", vote_count["fake"])
print("Final Result:", final_result)


In [ ]:
delete_extracted_frames(image_output_folder)

In [47]:
# image = Image.open("/content/drive/MyDrive/Colab Notebooks/Images_detection/face_0.jpg")

# # Call the predict function
# confidences, image_with_mask = predict(image)

# # Display the image with the mask overlay
# plt.imshow(image_with_mask)
# plt.title("Image with Mask Overlay")
# plt.axis('off')
# plt.show()

# # Print the confidence scores
# print("Confidence Scores:")
# print("Real:", confidences['real'])
# print("Fake:", confidences['fake'])


# Gradio Interface

In [ ]:
# interface = gr.Interface(
#     fn=predict,
#     inputs=[
#         gr.Image(label="Input Image", type="pil")
#     ],
#     outputs=[
#         gr.Label(label="Class"),
#         gr.Image(label="Face with Explainability", type="pil"),

#     ],
# ).launch()